In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import pymorphy2
from tqdm import tqdm, tqdm_notebook
import ssl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
tqdm().pandas()

[nltk_data] Downloading package punkt to /Users/16735244/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/16735244/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0it [00:00, ?it/s]/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
stopwords_en = stopwords.words("english")
stop = stopwords_en+stopwords_ru

morph = pymorphy2.MorphAnalyzer()
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stop:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    
    return tokens

0it [00:00, ?it/s]


In [3]:
#считывание данных
sub = pd.read_csv('train/sample_solution.csv', sep=';', index_col=None)
users = pd.read_csv('train/users.csv', sep=';', index_col=None, dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})
items = pd.read_csv('train/items.csv', sep=';', index_col=None, dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str})
train_transactions = pd.read_csv('train/train_transactions_extended.csv', sep=';', index_col=None, dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str})

In [4]:
train_transactions=train_transactions.sort_values(by=['chb', 'date_1']).reset_index()[['chb','sys_numb','date_1']]
items2 = items.fillna('')
items2['all']=items2['title']+' '+items2['author']+' '+items2['izd']+' '+items2['year_izd']
items2 = items2.reset_index(drop=True)
items2['index1'] = items2.index

In [5]:
items2['corpus']=items2['all'].progress_apply(lambda x : ' '.join(lemmatize(x)))

100%|██████████| 354355/354355 [15:30<00:00, 380.98it/s]


In [6]:
tfidf = TfidfVectorizer(max_features=70000)
tfidf_matrix = tfidf.fit_transform(items2['corpus'])

In [7]:
def coll(i=0):
    user_u = train_transactions['chb'].unique()
    lst=[]
    qwe=pd.DataFrame()
    while i<len(user_u):
        z = train_transactions[train_transactions['chb']==user_u[i]]['sys_numb']
        ind_book = items2[items2['sys_numb'].isin(z)]['index1']
        cosine_sim = linear_kernel(np.mean(tfidf_matrix[ind_book.values], axis=0), tfidf_matrix)
        x = pd.DataFrame(cosine_sim[0],columns= ['dist']).sort_values(by=['dist'])
        x['index1'] = x.index
        y = train_transactions[train_transactions['chb']==user_u[i]].sys_numb
        ind_now = items2[items2['sys_numb'].isin(y)].index1
        x = x[~x['index1'].isin(ind_now)][-20:].index.values
        lst = items2[items2['index1'].isin(x)]['sys_numb'].tolist()
        qwe = pd.concat([qwe,pd.DataFrame(lst)],ignore_index=True)
        i+=1
        #print(i)
    return qwe

In [8]:
qwe = coll()

In [9]:
zzz = sub.sort_values(by=['chb']).reset_index()
qwe['chb']=zzz['chb']
qwe = qwe.rename(columns={0: 'sys_numb'})

In [ ]:
qwe[['chb', 'sys_numb']].to_csv("solution01.csv", index=False, sep=';')